In [ ]:
"""Imports"""
import matplotlib.pyplot as plt
from matplotlib import style
import numpy as np
import os
import re
import os
import shutil
import sys
import pandas as pd
from io import StringIO




In [ ]:
"""Code block for getting directory paths, alg lists and test options"""

# Declaring global variables
kem_operations = ["keygen", "encaps", "decaps"]
sig_operations = ["keypair", "sign", "verify"]
kem_algs = []
sig_algs = []
system_type = ""
root_dir = ""
num_runs = 0
test_opts = [0,0]

#------------------------------------------------------------------------------
def get_system_type() :
    """Function for checking the system type and setting root_dir path"""
    global root_dir, system_type

    # Checking and storing system type
    if sys.platform == "win32":

        system_type = "win"
        root_dir = r"..\.."

    else:
        system_type = "linux"
        current_dir = os.getcwd()
        root_dir = os.path.dirname(os.path.dirname(current_dir))


#------------------------------------------------------------------------------*
def get_algs():
    """Function for creating list of algorithms"""

    # Checking liboqs version used to determine alg-list file
    flag_file = os.path.join(root_dir, "alg-lists", "version-flag.txt")

    with open(flag_file, "r") as flag:
        version_flag = flag.readline().strip()

    # Setting the alg list filename based on version flag
    if version_flag == "0":
        kem_algs_file = os.path.join(root_dir, "alg-lists", "kem-algs-v7.txt")
        sig_algs_file = os.path.join(root_dir, "alg-lists", "sig-algs-v7.txt")
    else:
        kem_algs_file = os.path.join(root_dir, "alg-lists", "kem-algs-v8.txt")
        sig_algs_file = os.path.join(root_dir, "alg-lists", "sig-algs-v8.txt")

    # # Getting the kem algs
    with open(kem_algs_file, "r") as kem_file:

        for line in kem_file:
            kem_algs.append(line.strip())
    
    # Getting the digital signature algorithms
    with open(sig_algs_file, "r") as alg_file:

        for line in alg_file:
            sig_algs.append(line.strip())

#------------------------------------------------------------------------------
def get_test_opts():
    """Function that will get the number of machines
        tested, and the number of test runs from the user
        and return a list variable"""

    # Loop for getting total machines from user
    while True:
        try:
            machine_num = int(input("Enter the number of machines tested - "))
            test_opts[0] = machine_num
            break
        except ValueError:
            print("Invalid Input - Please enter a number! - ")
    
    # Loop for getting total number of test runs from user
    while True:
        try:
            total_runs = int(input("Enter the number test runs - "))
            test_opts[1] = total_runs
            break
        except ValueError:
            print("Invalid Input - Please enter a number! - ")
    

#------------------------------------------------------------------------------
get_system_type()
get_algs()
get_test_opts()

In [ ]:
"""Code block for creating speed graphs for the machines"""

def get_excluded_algs(excluded_algs):
    """Excluded algoritms (used to filter which ones should appear in the grahp, leave exclude.txt empty for none)"""
    exlcude_filepath = os.path.join(root_dir, "result-processing", "graph-generators", "exclude.txt")

    # Gettting in algs to be exlcuded from file if any
    with open (exlcude_filepath, "r") as exclude_file:
        for line in exclude_file:
            excluded_algs.append(line.strip())

    return excluded_algs

#------------------------------------------------------------------------------*
def gen_graphs(machine_num, mach_results_dir):

    # Setting loop lists
    alg_type_list = ["KEM", "Digital Signature"]
    y_axis_labels = ["Number of Iterations", "Average Completion Time (s)", "CPU Cycles"]
    chart_titles = ["Iterations", "Time(s)", "CPU Cycles"]
    speed_metrics = ["Iterations", "Time (us): mean", "CPU cycles: mean"]
    axis_list_index = 0

    # Getting algs to be excluded if any
    excluded_algs = []
    excluded_algs = get_excluded_algs(excluded_algs)
    
    # Looping through each speed metric
    for speed_metric in speed_metrics:

        # Doing both kem and sig
        for alg_type in alg_type_list:

            # Setting filenames
            if alg_type == "KEM":
                speed_avg = os.path.join(mach_results_dir, "kem-speed-avg.csv")
            else:
                speed_avg = os.path.join(mach_results_dir, "sig-speed-avg.csv")

            #Importing ARM Iteration

            df_arm_og = pd.read_csv(speed_avg, index_col=False)
            df_arm_iteration = df_arm_og[["Algorithm","Operation",speed_metric]]

            # Excluding any algorithms supplised to exclude list if any
            df_arm_iteration = df_arm_iteration[~df_arm_iteration['Algorithm'].isin(excluded_algs)]

            #Assigning list values and removing trailing whitespace
            alg_list = df_arm_iteration['Algorithm'].unique().tolist()
            operation_list = (df_arm_iteration['Operation'].tolist())[:3]

            # Converting time measurement if that is current speed metric
            if "Time" in speed_metric:
                #df_arm_iteration.loc[:, speed_metric] = df_arm_iteration[speed_metric] / 1_000_000
                df_arm_iteration.loc[:, speed_metric] = df_arm_iteration[speed_metric] / 1_000
            
            kem_list_iteration = df_arm_iteration[speed_metric].tolist()

            #Setting bar parameters
            nump = np.arange(len(alg_list))
            width = 0.25
            plt.barh(nump-(width*2), kem_list_iteration[0::3], width, label=f'{operation_list[0].title()}', zorder=2)
            plt.barh(nump-width, kem_list_iteration[1::3], width, label=f'{operation_list[1].title()}', zorder=2)
            plt.barh(nump, kem_list_iteration[2::3], width, label=f'{operation_list[2].title()}', zorder=2)
            plt.yticks(nump - 0.25, alg_list)

            #Sytling the graph
            plt.rcParams["figure.autolayout"] = True
            plt.rcParams["figure.figsize"] = [10, 10]
            plt.grid(axis='y', linestyle='--', alpha=0.7, zorder=0)
            plt.grid(axis='x', linestyle='--', alpha=0.7, zorder=0)

            #Adding titles and plotting
            plt.xlabel(y_axis_labels[axis_list_index], labelpad=15, fontweight='bold')
            plt.ylabel("Algorithm Name", labelpad=25, fontweight='bold')
            plt.title(f"Machine {machine_num} - ARM CPU {alg_type} Benchmarking - {chart_titles[axis_list_index]}", fontweight='bold')
            plt.legend()
            plt.show()

        # Incrementing axis list index
        axis_list_index = axis_list_index + 1

def process_iteration_graphs():

    # Setting directory paths
    speed_result_root_dir = os.path.join(root_dir, "results", "liboqs", "speed-results")
    
    # Set number of machines variable from option list

    # Looping through each machine and generating iteration graphs
    for machine_num in range (1, (test_opts[0] + 1)):

        # Setting machine results dir directory path variable
        mach_results_dir = os.path.join(speed_result_root_dir, f"machine-{machine_num}")
        
        # Calling the graph gen functions for the current machine
        gen_graphs(machine_num, mach_results_dir)

process_iteration_graphs()